# Checking the symplecticity of a method

First, we need to import the packages that will be used in the following code.

In [1]:
import Pkg; Pkg.add("BSeries")
import Pkg; Pkg.add("RootedTrees")
import Pkg; Pkg.add("Latexify")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using BSeries
using RootedTrees
using Latexify

## The Interesting Part
Now I can start to define the functions needed for checking the symplecticity of a method.

The ideas mainly come from Sundklakk's master thesis and are only changed a little to translate it from python to julia.

First, I need a function that checks if the condition is satisfied for pair of trees of a specific order.

In [3]:
function SatisfiedForTreesOrder(condition, b, order)
    # Checking the condition for a pair of trees
    for order1 in range(1, order - 1)
        order2 = order - order1
        for tree1 in map(copy, RootedTreeIterator(order1))
            for tree2 in map(copy, RootedTreeIterator(order2))
                if condition(b, tree1, tree2) != true 
                    return false
                end
            end
        end
    end
    return true
end

SatisfiedForTreesOrder (generic function with 1 method)

Now I define the condition needed for the symplecticity.

In [4]:
function SymplecticityCondition(b::TruncatedBSeries, u, v)
    # Defines the condition needed for symplecticity as mentioned in my thesis
    return b[u∘v] + b[v∘u] == b[u] * b[v]
end

SymplecticityCondition (generic function with 1 method)

Finally, I can define the function that returns the symplecticity order of a method.

In [6]:
function SymplecticityOrder(b::TruncatedBSeries)
    symporder = 1

    # Checks if the method is consistent
    if b[rootedtree(Int64[])] != 1
        return 0
        
    else

        # This loops checks the symplecticity condition for a pair of trees of a given order
        for i in 1:order(b)
            in(rootedtree(collect(1:i)), keys(b))
            if SatisfiedForTreesOrder(SymplecticityCondition, b, i) == true                        
                symporder = i
            else 
                break 
            end
        end
    end
    return symporder
end

SymplecticityOrder (generic function with 1 method)

# Example
## Implicit Midpoint Method
A good example is the Gauß Method with 2 stages (Implicit Midpoint).
Since the method is not only pseudo-symplectic but indeed symplectic (means we are having a pseudo-symplecticity order $\infty$), we can choose the order of the B-series as high as we want to since the symplecticity order returned by the function will always be the same as the order of the B-series. 

In [29]:
A_im_mid = [1//2;;]
b_im_mid = [1]
c_im_mid = [1//2]
rk_im_mid = RungeKuttaMethod(A_im_mid, b_im_mid)
bseries_im_mid = bseries(rk_im_mid, 5)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 18 entries:
  RootedTree{Int64}: Int64[]         => 1//1
  RootedTree{Int64}: [1]             => 1//1
  RootedTree{Int64}: [1, 2]          => 1//2
  RootedTree{Int64}: [1, 2, 3]       => 1//4
  RootedTree{Int64}: [1, 2, 2]       => 1//4
  RootedTree{Int64}: [1, 2, 3, 4]    => 1//8
  RootedTree{Int64}: [1, 2, 3, 3]    => 1//8
  RootedTree{Int64}: [1, 2, 3, 2]    => 1//8
  RootedTree{Int64}: [1, 2, 2, 2]    => 1//8
  RootedTree{Int64}: [1, 2, 3, 4, 5] => 1//16
  RootedTree{Int64}: [1, 2, 3, 4, 4] => 1//16
  RootedTree{Int64}: [1, 2, 3, 4, 3] => 1//16
  RootedTree{Int64}: [1, 2, 3, 4, 2] => 1//16
  RootedTree{Int64}: [1, 2, 3, 3, 3] => 1//16
  RootedTree{Int64}: [1, 2, 3, 3, 2] => 1//16
  RootedTree{Int64}: [1, 2, 3, 2, 3] => 1//16
  RootedTree{Int64}: [1, 2, 3, 2, 2] => 1//16
  RootedTree{Int64}: [1, 2, 2, 2, 2] => 1//16

In [30]:
SymplecticityOrder(bseries_im_mid)

5

# Radau-IIA method with two stages

In [31]:
A_Radau2 = [5//12 -1//12; 3//4 1//4]
b_Radau2 = [3//4, 1//4]
c_Radau2 = [1//3, 1]
rk_Radau2 = RungeKuttaMethod(A_Radau2, b_Radau2)
bseries_Radau2 = bseries(rk_Radau2, 5)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 18 entries:
  RootedTree{Int64}: Int64[]         => 1//1
  RootedTree{Int64}: [1]             => 1//1
  RootedTree{Int64}: [1, 2]          => 1//2
  RootedTree{Int64}: [1, 2, 3]       => 1//6
  RootedTree{Int64}: [1, 2, 2]       => 1//3
  RootedTree{Int64}: [1, 2, 3, 4]    => 1//36
  RootedTree{Int64}: [1, 2, 3, 3]    => 1//18
  RootedTree{Int64}: [1, 2, 3, 2]    => 5//36
  RootedTree{Int64}: [1, 2, 2, 2]    => 5//18
  RootedTree{Int64}: [1, 2, 3, 4, 5] => -1//108
  RootedTree{Int64}: [1, 2, 3, 4, 4] => -1//54
  RootedTree{Int64}: [1, 2, 3, 4, 3] => 1//108
  RootedTree{Int64}: [1, 2, 3, 4, 2] => 1//27
  RootedTree{Int64}: [1, 2, 3, 3, 3] => 1//54
  RootedTree{Int64}: [1, 2, 3, 3, 2] => 2//27
  RootedTree{Int64}: [1, 2, 3, 2, 3] => 7//108
  RootedTree{Int64}: [1, 2, 3, 2, 2] => 7//54
  RootedTree{Int64}: [1, 2, 2, 2, 2] => 7//27

This method has order 3.

In [32]:
order_of_accuracy(bseries_Radau2)

3

The symplecticity order is the same as the order of consistency.

In [33]:
SymplecticityOrder(bseries_Radau2)

3

## Any other explicit method
This method is pseudo-symplectic of order $(2,4)$.

In [10]:
k = 3
A = [0 0 0;
(8*k-3)//(8k-4) 0 0;
(16k^2-8*k+1)//(2*k*(8k-3)) (2k-1)//(2*k*(8k-3)) 0]
b = [(3k-1)//(8k-3), (-2*(2k-1)^2)//(8k-3), k]
c = [0, (8k-3)//(8k-4), 1]
rkh = RungeKuttaMethod(A,b)
bser = bseries(rkh, 5)

TruncatedBSeries{RootedTree{Int64, Vector{Int64}}, Rational{Int64}} with 18 entries:
  RootedTree{Int64}: Int64[]         => 1//1
  RootedTree{Int64}: [1]             => 1//1
  RootedTree{Int64}: [1, 2]          => 1//2
  RootedTree{Int64}: [1, 2, 3]       => 1//8
  RootedTree{Int64}: [1, 2, 2]       => 3//8
  RootedTree{Int64}: [1, 2, 3, 4]    => 0//1
  RootedTree{Int64}: [1, 2, 3, 3]    => 21//160
  RootedTree{Int64}: [1, 2, 3, 2]    => 1//8
  RootedTree{Int64}: [1, 2, 2, 2]    => 39//160
  RootedTree{Int64}: [1, 2, 3, 4, 5] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 4] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 3] => 0//1
  RootedTree{Int64}: [1, 2, 3, 4, 2] => 0//1
  RootedTree{Int64}: [1, 2, 3, 3, 3] => 441//3200
  RootedTree{Int64}: [1, 2, 3, 3, 2] => 21//160
  RootedTree{Int64}: [1, 2, 3, 2, 3] => 1//192
  RootedTree{Int64}: [1, 2, 3, 2, 2] => 1//8
  RootedTree{Int64}: [1, 2, 2, 2, 2] => 339//3200

In [34]:
order_of_accuracy(bser)

2

In [11]:
SymplecticityOrder(bser)

4